In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType
import requests

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark DataFrames and SQL") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
hdfs_path = "hdfs://hdfs-nn:9000/AreasVerdes/bronze/edificiosVerdes_csv/edificiosVerdes.csv"

In [3]:
#Schema do dataset
customSchema = StructType([
    StructField("The_geom", StringType(), True),        
    StructField("Asset_Id", IntegerType(), True),
    StructField("GI_ID", StringType(), True),
    StructField("DEP_Contra", StringType(), True),
    StructField("DEP_Cont_1", IntegerType(), True),
    StructField("Project_Ty", StringType(), True),
    StructField("Row_Onsite", StringType(), True),
    StructField("Project_Na", StringType(), True),
    StructField("Asset_Type", StringType(), True),
    StructField("Status", StringType(), True),
    StructField("Asset_X_Co", FloatType(), True),
    StructField("Asset_Y_Co", FloatType(), True),
    StructField("Borough", StringType(), True),
    StructField("Sewer_Type", StringType(), True),
    StructField("Outfall", StringType(), True),
    StructField("Waterbody", StringType(), True),
    StructField("Street_Add", StringType(), True),
    StructField("Nearest_In", StringType(), True),
    StructField("BBL", LongType(), True),
    StructField("Secondary", StringType(), True),
    StructField("Community", IntegerType(), True),
    StructField("City_Counc", IntegerType(), True),
    StructField("Assembly_D", StringType(), True),
    StructField("Asset_leng", FloatType(), True),
    StructField("Asset_Widt", FloatType(), True),
    StructField("Asset_Area", FloatType(), True),
    StructField("GI_Feature", StringType(), True),
    StructField("Tree_Latin", StringType(), True),
    StructField("Tree_Commo", StringType(), True),
    StructField("Constructi", StringType(), True),
    StructField("Construc_1", StringType(), True),
    StructField("Status_Gro", StringType(), True),
])

episodes = spark \
            .read\
            .option("delimiter",",")\
            .option("header","true")\
            .schema(customSchema) \
            .csv(hdfs_path)

In [4]:
episodes.printSchema()
episodes.show()
#episodes.toPandas()

root
 |-- The_geom: string (nullable = true)
 |-- Asset_Id: integer (nullable = true)
 |-- GI_ID: string (nullable = true)
 |-- DEP_Contra: string (nullable = true)
 |-- DEP_Cont_1: integer (nullable = true)
 |-- Project_Ty: string (nullable = true)
 |-- Row_Onsite: string (nullable = true)
 |-- Project_Na: string (nullable = true)
 |-- Asset_Type: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Asset_X_Co: float (nullable = true)
 |-- Asset_Y_Co: float (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Sewer_Type: string (nullable = true)
 |-- Outfall: string (nullable = true)
 |-- Waterbody: string (nullable = true)
 |-- Street_Add: string (nullable = true)
 |-- Nearest_In: string (nullable = true)
 |-- BBL: long (nullable = true)
 |-- Secondary: string (nullable = true)
 |-- Community: integer (nullable = true)
 |-- City_Counc: integer (nullable = true)
 |-- Assembly_D: string (nullable = true)
 |-- Asset_leng: float (nullable = true)
 |-- Asset_Widt: 

In [5]:
# Escolho as colunas que quero guardar 
edificiosverdes = episodes.select("The_geom","GI_ID","DEP_Contra","Project_Ty","Row_Onsite","Project_Na","Asset_Type","Status","Borough","Sewer_Type","Outfall","Waterbody","Street_Add","Tree_Latin","Assembly_D","GI_Feature","Tree_Commo","Constructi","Status_Gro","Asset_Id","Community","City_Counc","Asset_X_Co","Asset_Y_Co","Asset_leng","Asset_Widt","Asset_Area","BBL")

In [6]:
#Altero os Not Found por Não temos essa informação na coluna Assembly_D
from pyspark.sql.functions import when
edificiosverdes = edificiosverdes.withColumn("Assembly_D", when(edificiosverdes.Assembly_D == "Not Found","Não temos informação") \
      .otherwise(edificiosverdes.Assembly_D))
edificiosverdes = edificiosverdes.na.fill("Não temos informação",["Assembly_D"])

In [7]:
#Altero os brancos por Não temos essa informação na coluna GI_Feature
from pyspark.sql.functions import when
edificiosverdes = edificiosverdes.withColumn("GI_Feature", when(edificiosverdes.GI_Feature == "","Não temos informação") \
      .otherwise(edificiosverdes.GI_Feature))
edificiosverdes = edificiosverdes.na.fill("Não temos informação",["GI_Feature"])

In [8]:
#Altero os brancos por Não temos essa informação na coluna Tree_Latin
from pyspark.sql.functions import when
edificiosverdes = edificiosverdes.withColumn("Tree_Latin", when(edificiosverdes.Tree_Latin == "","Não temos informação") \
      .otherwise(edificiosverdes.Tree_Latin))
edificiosverdes = edificiosverdes.na.fill("Não temos informação",["Tree_Latin"])

In [9]:
#Altero os brancos por Não temos essa informação na coluna Tree_Commo
from pyspark.sql.functions import when
edificiosverdes = edificiosverdes.withColumn("Tree_Commo", when(edificiosverdes.Tree_Commo == "","Não temos informação") \
      .otherwise(edificiosverdes.Tree_Commo))
edificiosverdes = edificiosverdes.na.fill("Não temos informação",["Tree_Commo"])

In [10]:
#Altero os brancos por Não temos essa informação na coluna Constructi
from pyspark.sql.functions import when
edificiosverdes = edificiosverdes.withColumn("Constructi", when(edificiosverdes.Constructi == "","Não temos informação") \
      .otherwise(edificiosverdes.Constructi))
edificiosverdes = edificiosverdes.na.fill("Não temos informação",["Constructi"])

In [11]:
#Altero os brancos por Não temos essa informação na coluna Status_Gro
from pyspark.sql.functions import when
edificiosverdes = edificiosverdes.withColumn("Status_Gro", when(edificiosverdes.Status_Gro == "","Não temos informação") \
      .otherwise(edificiosverdes.Status_Gro))
edificiosverdes = edificiosverdes.na.fill("Não temos informação",["Status_Gro"])

In [12]:
#Altero os brancos por Não temos essa informação na coluna Status_Gro
from pyspark.sql.functions import when
edificiosverdes = edificiosverdes.withColumn("Status_Gro", when(edificiosverdes.Tree_Commo == "","Não temos informação") \
      .otherwise(edificiosverdes.Tree_Commo))
edificiosverdes = edificiosverdes.na.fill("Não temos informação",["Tree_Commo"])

In [13]:
#Ir buscar o valor atual do euro em relação ao dolar através de uma API
requisicao = requests.get('https://economia.awesomeapi.com.br/all/USD-EUR')

cotacao = requisicao.json()
#Verificar se consegui ir buscar o valores corretos
cotacao

{'USD': {'code': 'USD',
  'codein': 'EUR',
  'name': 'Dólar Americano/Euro',
  'high': '0.9399',
  'low': '0.9334',
  'varBid': '-0.0033',
  'pctChange': '-0.35',
  'bid': '0.9344',
  'ask': '0.9348',
  'timestamp': '1672437593',
  'create_date': '2022-12-30 18:59:53'}}

In [14]:
#Conversão do valor em dolar para valor em euro utilizando uma API
edificiosverdes=edificiosverdes.withColumn("BBL", edificiosverdes.BBL * cotacao['USD']['bid'])

In [15]:
#Teste da conversão
edificiosverdes.createOrReplaceTempView("episodes")

sqlized_df = spark.sql(
    """
    SELECT Distinct BBL
    FROM episodes
    """
)

sqlized_df.show()

+--------------------+
|                 BBL|
+--------------------+
|   3.7663889826688E9|
|   2.8781295369344E9|
|   2.8796713016064E9|
|   3.7857776649344E9|
|    2.847200905344E9|
|2.8464347291136003E9|
|3.7812084508032002E9|
|   3.7834323274752E9|
|   2.8213086729344E9|
|   1.8966731622784E9|
|   3.7511581739008E9|
|   1.8988035849344E9|
|   1.8907210510976E9|
|   3.8459810774912E9|
|   3.8264241284736E9|
|   3.8223781129344E9|
|   2.8415197739008E9|
|   3.8402532064256E9|
|   3.8219950388352E9|
|     2.83837083936E9|
+--------------------+
only showing top 20 rows



In [16]:
#Conversão do valor em pés para valor em metros
edificiosverdes=edificiosverdes.withColumn("Asset_leng", edificiosverdes.Asset_leng * 0.3048)

In [17]:
#Conversão do valor em pés para valor em metros
edificiosverdes=edificiosverdes.withColumn("Asset_Widt", edificiosverdes.Asset_Widt * 0.3048)

In [18]:
#Conversão do valor em pés para valor em metros
edificiosverdes=edificiosverdes.withColumn("Asset_Area", edificiosverdes.Asset_Area * 0.3048)

In [19]:
edificiosverdes.printSchema()


root
 |-- The_geom: string (nullable = true)
 |-- GI_ID: string (nullable = true)
 |-- DEP_Contra: string (nullable = true)
 |-- Project_Ty: string (nullable = true)
 |-- Row_Onsite: string (nullable = true)
 |-- Project_Na: string (nullable = true)
 |-- Asset_Type: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Sewer_Type: string (nullable = true)
 |-- Outfall: string (nullable = true)
 |-- Waterbody: string (nullable = true)
 |-- Street_Add: string (nullable = true)
 |-- Tree_Latin: string (nullable = false)
 |-- Assembly_D: string (nullable = false)
 |-- GI_Feature: string (nullable = false)
 |-- Tree_Commo: string (nullable = false)
 |-- Constructi: string (nullable = false)
 |-- Status_Gro: string (nullable = false)
 |-- Asset_Id: integer (nullable = true)
 |-- Community: integer (nullable = true)
 |-- City_Counc: integer (nullable = true)
 |-- Asset_X_Co: float (nullable = true)
 |-- Asset_Y_Co: float (nullable = true)
 

In [20]:
#guardar
edificiosverdes \
    .select("The_geom","Asset_Id","GI_ID","DEP_ContrA","Project_Ty","Row_Onsite","Project_Na","Asset_Type","Status","Asset_X_Co","Asset_Y_Co","Borough","Sewer_Type","Outfall","Waterbody","BBL","Community","City_Counc","Assembly_D","Asset_leng","Asset_Widt","Asset_Area","GI_Feature","Tree_Latin", "Tree_Commo", "Constructi", "Status_Gro") \
    .write \
    .mode("overwrite") \
    .format("delta") \
    .save("hdfs://hdfs-nn:9000/AreasVerdes/silver/edificiosverdes")

In [21]:
#Testar se guardou
spark.sql(
    """
    SELECT GI_ID
    FROM AreasVerdes.edificiosverdes_table
    """
).show()

+-------+
|  GI_ID|
+-------+
|  1032A|
|  1032B|
|  1032C|
|  1032D|
|  1034A|
|  1040C|
|  1043A|
|  1046A|
|  1046B|
|  1249A|
|  1249B|
|  1249C|
|  1250A|
|  1250B|
|  1250C|
|  1250D|
|  1257A|
|1044-1A|
|1044-1B|
|1044-1C|
+-------+
only showing top 20 rows



In [22]:
spark.stop()